<a href="https://colab.research.google.com/github/katarinagresova/GLP/blob/main/experiments/CNN_for_DNA_Classification_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN for DNA Classification with PyTorch

## Introduction

This article is based on [A Complete Guide to CNN for Sentence Classification with PyTorch](https://chriskhanhtran.github.io/posts/cnn-sentence-classification/) and adapted to task of DNA classification.

**Convolutional Neural Networks (CNN)** were originally invented for computer vision (CV) and now are the building block of state-of-the-art CV models. One of the earliest applications of CNN in Natural Language Processing (NLP) was introduced in the paper ***Convolutional Neural Networks for Sentence Classification*** (Kim, 2014). With the same idea as in computer vision, CNN model is used as an feature extractor that encodes semantic features of sentences before these features are fed to a classifier.

With only a simple one-layer CNN trained on top of pretrained word vectors and little hyperparameter tuning, the model achieves excellent results on multiple sentence-level classification tasks. CNN models are now used widely other NLP tasks such as translation or question answering as a part of a more complex architecture.

CNNs also found their way into field of DNA classification. We can look at DNA as sequence of letters that are grouped into words and connected into sentences. So with just little effort we can adapt model created for sentence classification to classify DNA.


This article covers:
- Tokenizing and building vocabuilary from DNA data
- Loading pretrained dna2vec vectors and creating embedding layer for fine-tuning
- Building and training CNN model with PyTorch
- Advice for practitioners
- Bonus: Using Skorch as a scikit-like wrapper for PyTorch's Deep Learning models

**Reference:**
-  [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882) (Kim, 2014).
- [A Complete Guide to CNN for Sentence Classification with PyTorch](https://chriskhanhtran.github.io/posts/cnn-sentence-classification/) (Tran, 2020)
- [dna2vec: Consistent vector representations of variable-length k-mers](https://arxiv.org/abs/1701.06279) (Ng, 2017)



## 1. Set up

### 1.1. Install dependencies

In [4]:
!pip install genomic-benchmarks
!pip install gensim==4.1.2
!pip install nltk
!pip install torch
!pip install matplotlib
!pip install sklearn

You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/katarina/git/GLP/env/bin/python -m pip install --upgrade pip' command.


You will be promted to restart your environment in Google Colab after following cell. Just click on `RESTART RUNTIME` button at the bottom of output from following cell. Confirm the prompt and continue with executing cells.

In [5]:
!wget https://raw.githubusercontent.com/xinrcornelia/dna2vec/master/requirements.txt
!pip install -r requirements.txt

--2022-02-01 11:08:18--  https://raw.githubusercontent.com/xinrcornelia/dna2vec/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 351 [text/plain]
Saving to: ‘requirements.txt.3’

requirements.txt.3  100%[===================>]     351  --.-KB/s    in 0s      

2022-02-01 11:08:18 (7,92 MB/s) - ‘requirements.txt.3’ saved [351/351]

  Using cached biopython-1.68-cp38-cp38-linux_x86_64.whl
  Using cached python_dateutil-2.8.0-py2.py3-none-any.whl (226 kB)
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2
  Attempting uninstall: biopython
    Found existing installation: biopython 1.79
    Uninstalling bi

### 1.2. Import Libraries

In [6]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
import torch

%matplotlib inline

[nltk_data] Downloading package punkt to /home/katarina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.3. Download Dataset

The dataset we will use is Human Nontata Promotes, genomic dataset from [Genomic Benchmarks](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks). The dataset has 19657 positive and 16474 negative sentences with length of 251 characters.

In [184]:
from genomic_benchmarks.loc2seq import download_dataset
data_path = download_dataset("demo_mouse_enhancers", version=0)
data_path

Unzipping...Done.


PosixPath('/home/katarina/.genomic_benchmarks/demo_mouse_enhancers')

In [8]:
from genomic_benchmarks.data_check import info
info("demo_mouse_enhancers", version=0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


### 1.4. Load Data

Loading training data into `np.array`. Downloaded data have each sequence stored in a separate `.txt` file. We also insert spaces into sequences to create words of fixed length. This way, we can use word tokenization later.

In [185]:
WORD_LENGTH = 3

def insert_spaces(text, length):

    # strip last word that would be shorter than 'length'
    text = text[: -1 * (len(text) % length)]

    # split 'text' into words of length 'length' - insert space between words
    return ' '.join(text[i:i+length] for i in range(0,len(text),length))

def read_text(data_dir):
    """Load text data, uppercase text and save to a list."""

    texts = []
    for filename in os.listdir(data_dir):
        with open(os.path.join(data_dir, filename), 'r') as f: # open in readonly mode
            for line in f:
                texts.append(insert_spaces(line.upper().strip(), WORD_LENGTH))

    return texts

def load_data(data_dir, train=True):

    if train:
        folder = 'train'
    else:
        folder = 'test'

    # Load files
    neg_text = read_text(data_dir / folder / 'negative/')
    pos_text = read_text(data_dir / folder / 'positive/')

    # Concatenate and label data
    texts = np.array(neg_text + pos_text)
    labels = np.array([0]*len(neg_text) + [1]*len(pos_text))

    return texts, labels

In [186]:
texts, labels = load_data(data_path, train=True)

Let's look at one training sequence.

In [140]:
texts[0]

'TAG TCC CAG CTA CTT GAG AGG CTG AGG TGG GAG GAT CAC TTG AGC CTG GGA GGT TGA GGC TGC AGT GAG CCG TGA TTG TGC CAC TGC ACT TCA GCC TGG GCA ACA CAG TGA GAC CTC ATC TCT AAA AGA AGA AGA AAA AAA AAG GAA ATC CTG CAA TGT GTG ACA ACA TGA ATA AAC TCT GAG GAA ATA ATG CTA AGT GAA GTA AGC CAG GGA CAG GAC AAA TAC CGC ATG ATT CCA CCT ATA GAA GGA'

### 1.5. Prepare dna2vec vectors

Here we use dna2vec: Consistent vector representations of variable-length k-mers (word2vec adapted for DNA).



In [11]:
!wget https://github.com/pnpnpn/dna2vec/raw/master/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v

--2022-02-01 11:08:57--  https://github.com/pnpnpn/dna2vec/raw/master/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pnpnpn/dna2vec/master/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v [following]
--2022-02-01 11:08:58--  https://raw.githubusercontent.com/pnpnpn/dna2vec/master/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84042299 (80M) [text/plain]
Saving to: ‘dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v.

This is extracted part of dna2vev implementation modified by [xinrcornelia](https://github.com/xinrcornelia). Original implementation was using old and uncompatible libraries. 

In [177]:
#Thanks to https://github.com/xinrcornelia/dna2vec/blob/master/dna2vec/multi_k_model.py
from gensim.models import KeyedVectors
import tempfile

class SingleKModel:
    def __init__(self, model):
        self.model = model
        self.vocab_lst = sorted(model.index_to_key)

class MultiKModel:
    def __init__(self, filepath):
        self.aggregate = KeyedVectors.load_word2vec_format(filepath, binary=False)
        #self.logger = logbook.Logger(self.__class__.__name__)

        vocab_lens = [len(vocab) for vocab in self.aggregate.index_to_key]
        self.k_low = min(vocab_lens)
        self.k_high = max(vocab_lens)
        self.vec_dim = self.aggregate.vector_size

        self.data = {}
        for k in range(self.k_low, self.k_high + 1):
            self.data[k] = self.separate_out_model(k)

    def vector(self, vocab):
        return self.data[len(vocab)].model[vocab]

    def l2_norm(self, vocab):
        return np.linalg.norm(self.vector(vocab))

    def separate_out_model(self, k_len):
        vocabs = [vocab for vocab in self.aggregate.index_to_key if len(vocab) == k_len]
        if len(vocabs) != 4 ** k_len:
            self.logger.warn('Missing {}-mers: {} / {}'.format(k_len, len(vocabs), 4 ** k_len))

        header_str = '{} {}'.format(len(vocabs), self.vec_dim)
        with tempfile.NamedTemporaryFile(mode='w') as fptr:
            print(header_str, file=fptr)
            for vocab in vocabs:
                vec_str = ' '.join("%f" % val for val in self.aggregate[vocab])
                print('{} {}'.format(vocab, vec_str), file=fptr)
            fptr.flush()
            return SingleKModel(KeyedVectors.load_word2vec_format(fptr.name, binary=False))

In [178]:
filepath = 'dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v'
mk_model = MultiKModel(filepath)

Each DNA word is represented by vector with 100 dimensions.

In [14]:
mk_model.vector('AGT')

array([-0.100437,  0.037371, -0.003308, -0.17211 ,  0.263888,  0.213367,
        0.139173, -0.228796, -0.43588 , -0.152078, -0.06351 ,  0.12935 ,
       -0.309992,  0.110419,  0.15974 ,  0.422234,  0.11112 , -0.152604,
       -0.194822, -0.08244 ,  0.146237,  0.162316, -0.056551,  0.226918,
       -0.566757, -0.160063, -0.476681,  0.246377,  0.402231,  0.49138 ,
        0.148335, -0.276922,  0.306808, -0.256018, -0.023064,  0.23046 ,
       -0.370285,  0.353037, -0.116914,  0.465472,  0.168274,  0.347329,
        0.321257, -0.034686,  0.03985 ,  0.123678, -0.148329, -0.216877,
       -0.061961, -0.132535, -0.090232, -0.301371, -0.264962, -0.030402,
        0.107969, -0.09688 ,  0.133854, -0.43233 ,  0.130724, -0.203363,
       -0.014014,  0.063082,  0.023929,  0.067139,  0.22355 ,  0.158326,
        0.10423 ,  0.02596 ,  0.114201, -0.139774, -0.349662, -0.12374 ,
       -0.328185,  0.079055, -0.22557 , -0.106094, -0.244369, -0.405145,
       -0.144079,  0.052439, -0.031427,  0.036935, 

### 1.6. Set up GPU for Training

Google Colab offers free GPUs and TPUs. Since we'll be training a large neural network it's best to utilize these features.

A GPU can be added by going to the menu and selecting:

> Runtime -> Change runtime type -> Hardware accelerator: GPU

Then we need to run the following cell to specify the GPU as the device.

In [15]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## 2. Data Preparation

To prepare our text data for training, first we need to tokenize our sentences and build a vocabulary dictionary `word2idx`, which will later be used to convert our tokens into indexes and build an embedding layer.

***So, what is an embedding layer?***

An embedding layer serves as a look-up table which take word indexes in the vocabulary as input and output word vectors. Hence, the embedding layer has shape $(N, d)$ where $N$ is the size of the vocabulary and $d$ is the embedding dimension. In order to fine-tune pretrained word vectors, we need to create an embedding layer in our `nn.Modules` class. Our input to the model will then be `input_ids`, which is the tokens' index in the vocabulary.

### 2.1. Tokenize

The function `tokenize` will tokenize our sentences, build a vocabulary and fine the maximum sentence length. The function `encode` will take in the outputs of `tokenize`, perform sentence padding and return `input_ids` as a numpy array.

In [144]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def build_vocab(texts):
    """Build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return word2idx, max_len

def tokenize(texts):
    tokenized_texts = []
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)
    return tokenized_texts

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:

        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = []
        for token in tokenized_sent:
            if token in word2idx:
                input_id.append(word2idx.get(token))
            else:
                input_id.append(word2idx.get('<unk>'))
        input_ids.append(input_id)
    
    return np.array(input_ids)

### 2.2. Load Pretrained Vectors

We will load the pretrain vectors for each tokens in our vocabulary. For tokens with no pretraiend vectors, we will initialize random word vectors with the same length and variance.

In [142]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    d = len(mk_model.vector('AAA'))

    # Initialize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    # Initialize fixed embeddings for special characters
    # embeddings[word2idx['<pad>']] = np.zeros((d,))
    # embeddings[word2idx['<unk>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for word in word2idx.keys():
        try:
          embeddings[word2idx[word]] = mk_model.vector(word)
        except:
          pass

    return embeddings

Now let's put above steps together.

In [187]:
# Tokenize, build vocabulary, encode tokens
word2idx, max_len = build_vocab(texts)
tokenized_texts = tokenize(texts)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx)
embeddings = torch.tensor(embeddings)

In [188]:
input_ids

array([[ 2,  3,  4, ...,  0,  0,  0],
       [50, 50, 50, ...,  0,  0,  0],
       [50, 50, 50, ...,  0,  0,  0],
       ...,
       [17, 55, 54, ...,  0,  0,  0],
       [21, 35, 18, ...,  0,  0,  0],
       [23, 43, 35, ...,  0,  0,  0]])

In [32]:
embeddings[word2idx['<pad>']]

tensor([ 0.0167,  0.0482, -0.0670,  0.1536,  0.0116, -0.1916, -0.1855,  0.0184,
         0.2183, -0.0958,  0.1848,  0.1320, -0.1261, -0.2318, -0.1991,  0.1142,
        -0.1502, -0.0644,  0.0008,  0.1127, -0.1934, -0.0468, -0.0075,  0.0204,
         0.1592, -0.1642,  0.1539,  0.0964,  0.0279,  0.0998, -0.0440, -0.1808,
         0.0829,  0.0123,  0.0697, -0.0810,  0.2034,  0.0437, -0.2073, -0.0251,
         0.1282,  0.1471, -0.0027,  0.1974, -0.1162, -0.0447,  0.1993,  0.1981,
         0.0929,  0.1795, -0.2056,  0.1825, -0.0897, -0.1608, -0.1303, -0.0410,
        -0.1244, -0.0897, -0.1825, -0.1019,  0.0361, -0.0291, -0.0766,  0.1232,
         0.1968,  0.1284, -0.0840,  0.1222, -0.1740, -0.1848, -0.0772,  0.1539,
        -0.1268, -0.2184, -0.1841,  0.1794,  0.0669, -0.1234,  0.0276,  0.0138,
         0.0777, -0.1334,  0.1822, -0.0953,  0.0971, -0.2419, -0.0637, -0.0741,
         0.0217, -0.2136,  0.1451, -0.1411,  0.0885,  0.1516,  0.0309,  0.1204,
         0.0809, -0.0722,  0.1544, -0.10

In [33]:
embeddings[word2idx['ACG']]

tensor([-0.0675,  0.3851, -0.0974, -0.0475, -0.0506, -0.0216, -0.3819,  0.0047,
        -0.0318, -0.4523, -0.3082,  0.1640,  0.4856, -0.2600, -0.4911,  0.1989,
        -0.0043,  0.0260, -0.1499,  0.4140,  0.0644, -0.2304,  0.1913,  0.1899,
        -0.1383, -0.1455, -0.3562,  0.3087,  0.3917,  0.0984,  0.1479, -0.3244,
         0.2707, -0.6842,  0.5716,  0.3798,  0.1368,  0.5149,  0.3511,  0.3752,
         0.7564,  0.0129, -0.0028, -0.0422, -0.2850,  0.1929, -0.1624, -0.0447,
         0.0557, -0.1682,  0.2852,  0.1733, -0.1773, -0.2135, -0.1532,  0.2159,
         0.0317, -0.1270,  0.2978,  0.0668,  0.4469, -0.0740, -0.1202,  0.5711,
         0.1423,  0.3321,  0.0894,  0.1014,  0.2305,  0.0294, -0.1397,  0.2471,
        -0.1196, -0.0268,  0.1059, -0.4370, -0.2137,  0.1215, -0.5145,  0.1736,
        -0.2941,  0.1140, -0.1546, -0.0839,  0.0507, -0.5224, -0.1583, -0.3286,
         0.2901,  0.2326, -0.4448,  0.0074,  0.1123,  0.1167,  0.0676, -0.0183,
         0.0072,  0.1639, -0.3864, -0.37

### 2.3. Create PyTorch DataLoader

We will create an iterator for our dataset using the torch DataLoader class. This will help save on memory during training and boost the training speed. The batch_size used in the paper is 50.

In [146]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(inputs, labels, batch_size=50):
    """Convert data sets to torch.Tensors and load it to DataLoader.
    """

    # Convert data type to torch.Tensor
    inputs, labels = tuple(torch.tensor(data) for data in [inputs, labels])

    # Create DataLoader for  data
    tensor_data = TensorDataset(inputs, labels)
    sampler = RandomSampler(tensor_data)
    dataloader = DataLoader(tensor_data, sampler=sampler, batch_size=batch_size)

    return dataloader

We will use 90% of the dataset for training and 10% for validation.

In [147]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader = data_loader(train_inputs, train_labels)
val_dataloader = data_loader(val_inputs, val_labels)

## 3. Model

**CNN Architecture**

The picture below is the illustration of the CNN architecture that we are going to build with three filter sizes: 2, 3, and 4, each of which has 2 filters.

![](https://github.com/chriskhanhtran/CNN-Sentence-Classification-PyTorch/blob/master/cnn-architecture.JPG?raw=true)

*CNN Architecture (Source: Zhang, 2015)*

```python
# Sample configuration:
filter_sizes = [2, 3, 4]
num_filters = [2, 2, 2]
```

Suppose that we are classifying the sentence "***I like this movie very much!***" ($N = 7$ tokens) and the dimensionality of word vectors is $d=5$. After applying the embedding layer on the input token ids, the sample sentence is presented as a 2D tensor with shape (7, 5) like an image.

$$\mathrm{x_{emb}} \quad \in \mathbb{R}^{7 \times 5}$$

We then use 1-dimesional convolution to extract features from the sentence. In this example, we have 6 filters in total, and each filter has shape $(f_i, d)$ where $f_i$ is the filter size for $i \in \{1,...,6\}$. Each filter will then scan over $\mathrm{x_{emb}}$ and returns a feature map:

$$\mathrm{x_{conv_ i} = Conv1D(x_{emb})} \quad \in \mathbb{R}^{N-f_i+1}$$

Next, we apply the ReLU activation to $\mathrm{x_{conv_{i}}}$ and use max-over-time-pooling to reduce each feature map to a single scalar. Then we concatenate these scalars into the final feature vector which will be fed to a fully connected layer to compute the final scores for our classes (logits).

$$\mathrm{x_{pool_i} = MaxPool(ReLU(x_{conv_i}))} \quad \in \mathbb{R}$$

$$\mathrm{x_{fc} = \texttt{concat}(x_{pool_i})} \quad \in \mathbb{R}^6$$

The idea here is that each filter will capture different semantic signals in the sentence (ie. happiness, humor, politic, anger...) and max-pooling will record only the strongest signal over the sentence. This logic makes sense because humans also perceive the sentiment of a sentence based on its strongest word/signal.

Finally, we use a fully connected layer with the weight matrix $\mathbf{W_{fc}} \in \mathbb{R}^{2 \times 6} $ and dropout to compute $\mathrm{logits}$, which is a vector of length 2 that keeps the scores for 2 classes.

$$\mathrm{logits = Dropout(\mathbf{W_{fc}}x_{fc})}  \in \mathbb{R}^2$$

An in-depth explanation of CNN can be found in this [article](https://cs231n.github.io/convolutional-networks/) and this [video](https://www.youtube.com/watch?v=YRhxdVk_sIs).










### 3.1. Create CNN Model

For simplicity, the model above has very small configurations. The final model we'll use is much bigger but has the same architecture:

|Description         |Values           |
|:------------------:|:---------------:|
|input word vectors  |dna2vec         |
|embedding size      |100              |
|filter sizes        |(3, 4, 5)        |
|num filters         |(100, 100, 100)  |
|activation          |ReLU             |
|pooling             |1-max pooling    |
|dropout rate        |0.5              |



In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_DNA(nn.Module):
    """An 1D Convulational Neural Network for DNA Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=100,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_DNA class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_DNA, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

### 3.2. Optimizer

To train Deep Learning models, we need to define a loss function and minimize this loss. We'll use back-propagation to compute gradients and use an optimization algorithm (ie. Gradient Descent) to minimize the loss. The original paper used the Adadelta optimizer.

In [160]:
import torch.optim as optim

def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_DNA(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer

### 3.3. Training Loop

For each epoch, the code below will perform a forward step to compute the *Cross Entropy* loss, a backward step to compute gradients and use the optimizer to update weights/parameters. At the end of each epoch, the loss on training data and the accuracy over the validation data will be printed to help us keep track of the model's performance. The code is heavily annotated with detailed explanations.

In [38]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

## 4. Evaluation 

In the original paper (Kim, 2014), the author tried different variations of the model.
- **CNN-rand**: The baseline model where the embedding layer is randomly initialized and then updated during training.
- **CNN-static**: A model with pretrained vectors. However, the embedding layer is freezed during training.
- **CNN-non-static**: Same as above but the embedding layers are fine-tuned during training.

Tran,2020 did experiments with all 3 variations and compared their performance. Below is the report of results.

|Model            |Kim's results  |Tran's results  |
|:----------------|:-------------:|:-----------:|
|CNN-rand         |76.1           |74.2         |
|CNN-static       |81.0           |82.7         |
|CNN-non-static   |81.5           |84.4         |

Randomness could cause the difference in the results. Improvement in Tran's results could be caused by using fastText pretrained vectors, which are of higher quality than word2vec vectors that Kim used.

In this notebook, we adapted model proposed by Tran,2020 to use with DNA sequences and we run similar experiment with pretrained dna2vec embedding vectors. Below is the report of results.

|Model            |Our results    |
|:----------------|:-------------:|
|CNN-rand         |76.1           |
|CNN-static       |81.0           |
|CNN-non-static   |81.5           |

Another benefit of pretrained embeddings is speedup of model training. One epoch of training is about 2 times slower when we need to compute embedding vectors.


First, we will run CNN model with random embeddings that we will try to learn during training. We are using `embed_dim=100` to match dimensions of `dna2vec` embeddings used in next models.

In [162]:
# CNN-rand: Word vectors are randomly initialized.
set_seed(42)
cnn_rand, optimizer = initilize_model(vocab_size=len(word2idx),
                                      embed_dim=100,
                                      learning_rate=0.3,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   0.497624   |  0.442190  |   79.31   |   16.46  
   2    |   0.447060   |  0.435506  |   79.60   |   12.77  
   3    |   0.419435   |  0.408050  |   81.02   |   12.75  
   4    |   0.394140   |  0.387862  |   82.04   |   13.46  
   5    |   0.365410   |  0.392974  |   80.65   |   11.85  
   6    |   0.339719   |  0.376152  |   82.65   |   13.39  
   7    |   0.309033   |  0.344586  |   85.02   |   15.62  
   8    |   0.279932   |  0.345872  |   84.84   |   11.65  
   9    |   0.254501   |  0.343436  |   85.09   |   10.63  
  10    |   0.242257   |  0.327282  |   86.15   |   10.61  
  11    |   0.222653   |  0.330655  |   86.11   |   10.02  
  12    |   0.208677   |  0.334777  |   85.78   |   10.24  
  13    |   0.193446   |  0.321205  |   85.89   |   10.79  
  14    |   0.183274   |  0.345823  |   86.22   |   12.63  
  15    |   0.166617

In [163]:
# CNN-static: dna2vec pretrained word vectors are used and freezed during training.
set_seed(42)
cnn_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                        freeze_embedding=True,
                                        learning_rate=0.3,
                                        dropout=0.5)
train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   0.496641   |  0.453007  |   78.80   |   8.82   
   2    |   0.434299   |  0.417284  |   80.55   |   8.25   
   3    |   0.399890   |  0.439137  |   80.22   |   11.71  
   4    |   0.377305   |  0.383808  |   82.51   |   7.97   
   5    |   0.351677   |  0.374156  |   82.65   |   10.01  
   6    |   0.328584   |  0.366556  |   82.87   |   8.86   
   7    |   0.306771   |  0.354151  |   83.78   |   8.53   
   8    |   0.286681   |  0.376002  |   83.82   |   8.77   
   9    |   0.270412   |  0.358195  |   84.80   |   7.55   
  10    |   0.251574   |  0.396317  |   83.56   |   7.46   
  11    |   0.235887   |  0.346546  |   85.13   |   7.93   
  12    |   0.218733   |  0.328022  |   85.78   |   7.69   
  13    |   0.208285   |  0.367171  |   85.20   |   8.05   
  14    |   0.197947   |  0.331507  |   86.62   |   7.62   
  15    |   0.185154

In [164]:
# CNN-non-static: dna2vec pretrained word vectors are fine-tuned during training.
set_seed(42)
cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.3,
                                            dropout=0.5)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=20)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   0.493290   |  0.447149  |   79.16   |   11.07  
   2    |   0.432897   |  0.415333  |   80.58   |   12.04  
   3    |   0.398694   |  0.415508  |   80.98   |   11.01  
   4    |   0.374678   |  0.382646  |   82.76   |   11.90  
   5    |   0.346559   |  0.368753  |   83.02   |   11.23  
   6    |   0.320976   |  0.370778  |   82.25   |   10.95  
   7    |   0.297033   |  0.353070  |   83.53   |   10.91  
   8    |   0.273787   |  0.370409  |   84.29   |   10.51  
   9    |   0.255535   |  0.358721  |   84.76   |   10.13  
  10    |   0.233931   |  0.354662  |   84.84   |   10.51  
  11    |   0.217920   |  0.360115  |   85.31   |   10.34  
  12    |   0.200771   |  0.412289  |   84.33   |   10.38  
  13    |   0.188879   |  0.329183  |   86.22   |   11.32  
  14    |   0.175321   |  0.353973  |   85.71   |   10.60  
  15    |   0.168692

## 5. Test Models

We have left out part of data for testing. We will preprocess it and use it for testing, how will our models perform on data they never seen.

In [189]:
test_texts, test_labels = load_data(data_path, train=False)

tokenized_test_texts = tokenize(test_texts)
max_len = max(max_len, max([len(seq) for seq in tokenized_test_texts]))
test_input_ids = encode(tokenized_test_texts, word2idx, max_len)

In [166]:
test_input_ids

array([[52,  3, 54, ..., 52,  6,  9],
       [24, 45,  6, ..., 41, 56, 50],
       [ 2, 55, 32, ..., 35, 50, 34],
       ...,
       [53, 46, 25, ..., 10,  4, 62],
       [50, 22, 61, ..., 50, 14, 62],
       [12, 37, 50, ..., 49, 17, 55]])

In [167]:
test_dataloader = data_loader(test_input_ids, test_labels)

In [168]:
def test(model, dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()


    val_accuracy = []
    val_loss = []

    # Tracking variables
    size = dataloader.dataset.__len__()
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tp, p, fp, tn, fn = 0, 0, 0, 0, 0
    accuracy = 0
    correct = 0

    # For each batch in our validation set...
    for batch in dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Compute loss
        loss = loss_fn(logits, b_labels)
        test_loss += loss.item()

        # Compute number of correct predictions
        correct += (preds == b_labels).sum().item()

        # Calculate the accuracy rate
        accuracy += (preds == b_labels).cpu().numpy().mean() * 100

        # Compute number of positive predictions
        p += (b_labels == 1).sum().item()

        # Compute true positives and false positives
        tp += (b_labels * preds).sum(dim=0).item()
        fp += ((1 - b_labels) * preds).sum(dim=0).item()

        # Compute true negatives and false negatives
        tn += ((1 - b_labels) * (1 - preds)).sum(dim=0).item()
        fn += (b_labels * (1 - preds)).sum(dim=0).item()

    print("p ", p, "; tp ", tp, "; fp ", fp, "; tn ", tn, "; fn ", fn)
    recall = tp / p
    precision = tp / (tp + fp)
    print("recall ", recall, "; precision ", precision)
    f1_score = 2 * precision * recall / (precision + recall)
    
    print("num_batches", num_batches)
    print("correct", correct)
    print("size", size)

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test metrics: \n Accuracy: {accuracy:>6f}, F1 score: {f1_score:>6f}, Avg loss: {test_loss:>6f} \n")
    
    return accuracy, f1_score

In [169]:
test(model=cnn_rand.to("cpu"), dataloader=test_dataloader)

p  4915 ; tp  3977 ; fp  227 ; tn  3892 ; fn  938
recall  0.8091556459816887 ; precision  0.9460038058991437
num_batches 181
correct 7869
size 9034
Test metrics: 
 Accuracy: 0.871043, F1 score: 0.872245, Avg loss: 0.346019 



(0.8710427274739871, 0.8722447636802281)

In [170]:
test(model=cnn_static.to("cpu"), dataloader=test_dataloader)

p  4915 ; tp  4107 ; fp  476 ; tn  3643 ; fn  808
recall  0.8356052899287895 ; precision  0.8961379009382501
num_batches 181
correct 7750
size 9034
Test metrics: 
 Accuracy: 0.857870, F1 score: 0.864814, Avg loss: 0.570633 



(0.8578702678769095, 0.86481364497789)

In [171]:
test(model=cnn_non_static.to("cpu"), dataloader=test_dataloader)

p  4915 ; tp  4370 ; fp  493 ; tn  3626 ; fn  545
recall  0.8891149542217701 ; precision  0.8986222496401398
num_batches 181
correct 7996
size 9034
Test metrics: 
 Accuracy: 0.885101, F1 score: 0.893843, Avg loss: 0.293465 



(0.8851007305733894, 0.8938433217426877)

## Hyper parameters tuning

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

## Trying it on more datasets

In [191]:
# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx)
embeddings = torch.tensor(embeddings)

table = []

# using only datasets where classes are called positive and negative

datasets = ['demo_mouse_enhancers', 'human_enhancers_cohn', 'human_enhancers_ensembl', 'human_nontata_promoters']

for dataset in datasets:

    print('DATASET: ', dataset)

    data_path = download_dataset(dataset, version=0)
    texts, labels = load_data(data_path, train=True)

    # Tokenize, build vocabulary, encode tokens
    word2idx, max_len = build_vocab(texts)
    tokenized_texts = tokenize(texts)
    input_ids = encode(tokenized_texts, word2idx, max_len)

    # Load pretrained vectors
    embeddings = load_pretrained_vectors(word2idx)
    embeddings = torch.tensor(embeddings)

    # Train Test Split
    train_inputs, val_inputs, train_labels, val_labels = train_test_split(
        input_ids, labels, test_size=0.1, random_state=42)

    # Load data to PyTorch DataLoader
    train_dataloader = data_loader(train_inputs, train_labels)
    val_dataloader = data_loader(val_inputs, val_labels)

    # CNN-rand: Word vectors are randomly initialized.
    set_seed(42)
    print('RANDOM')
    cnn_rand, optimizer = initilize_model(vocab_size=len(word2idx),
                                        embed_dim=100,
                                        learning_rate=0.3,
                                        dropout=0.5)
    train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=20)

    # CNN-static: dna2vec pretrained word vectors are used and freezed during training.
    set_seed(42)
    print('STATIC')
    cnn_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=True,
                                            learning_rate=0.3,
                                            dropout=0.5)
    train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=20)

    # CNN-non-static: dna2vec pretrained word vectors are fine-tuned during training.
    set_seed(42)
    print('NON STATIC')
    cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                                freeze_embedding=False,
                                                learning_rate=0.3,
                                                dropout=0.5)
    train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=20)

    print('GOING TO TEST')
    test_texts, test_labels = load_data(data_path, train=False)
    tokenized_test_texts = tokenize(test_texts)
    max_len = max(max_len, max([len(seq) for seq in tokenized_test_texts]))
    test_input_ids = encode(tokenized_test_texts, word2idx, max_len)
    test_dataloader = data_loader(test_input_ids, test_labels)

    cnn_rand_acc, cnn_rand_f1 = test(model=cnn_rand.to("cpu"), dataloader=test_dataloader)
    cnn_static_acc, cnn_static_f1 = test(model=cnn_static.to("cpu"), dataloader=test_dataloader)
    cnn_non_static_acc, cnn_non_static_f1 = test(model=cnn_static.to("cpu"), dataloader=test_dataloader)

    table.append([dataset, cnn_rand_f1, cnn_static_f1, cnn_non_static_f1])

pd.DataFrame(table, columns=['dataset', 'cnn_rand_f1', 'cnn_static_f1', 'cnn_non_static_f1'])

DATASET:  demo_mouse_enhancers
Unzipping...Done.
RANDOM
Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------
   1    |   0.716757   |  0.679567  |   49.47   |   6.14   
   2    |   0.678335   |  0.626182  |   64.91   |   6.15   
   3    |   0.653161   |  0.695404  |   38.89   |   6.04   
   4    |   0.659598   |  0.626203  |   56.60   |   6.35   
   5    |   0.646923   |  0.620921  |   61.79   |   6.04   
   6    |   0.592952   |  0.585424  |   63.53   |   5.98   
   7    |   0.593763   |  0.586397  |   68.11   |   6.06   
   8    |   0.578219   |  0.602496  |   59.91   |   6.11   
   9    |   0.580130   |  0.603331  |   60.66   |   5.91   
  10    |   0.555025   |  0.624593  |   53.53   |   5.90   
  11    |   0.562874   |  0.567810  |   63.85   |   5.98   
  12    |   0.544354   |  0.565428  |   67.17   |   6.00   
  13    |   0.529997   |  0.616676  |   56.53   |   5.85   
  14    |   0.524942   |

,dataset,cnn_rand_f1,cnn_static_f1,cnn_non_static_f1
0,demo_mouse_enhancers,0.494382,0.720280,0.720280
1,human_enhancers_cohn,0.694113,0.727552,0.727552
2,human_enhancers_ensembl,0.644330,0.592812,0.592812
3,human_nontata_promoters,0.872245,0.864814,0.864814


## 6. Advice for Practitioners

In [***A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification***](https://arxiv.org/abs/1510.03820) (Zhang, 2015), the authors conducted a sensitivity analysis of the above CNN architecture by running it many different sets of hyperparameters. Based on main empirical findings of the research, below are some advice for practioners to choose hyperparameters when applying this architecture for sentence classification tasks:
- **Input word vectors:** Using pretrained word vectors such as word2vec, Glove (or fastText in our implementation) yields much better results than using one-hot vectors or randomly initialized vectors.
- **Filter region size** can have a large effect on performance, and should be tuned. A reasonable range might be 1~10. For example, using `filter_size=[7]` and `num_filters=[400]` yields the best result in the MR dataset.
- **Number of feature maps:** try values from 100 to 600 for each filter region size.
- **Activation funtions:** ReLu and tanh are the best candidates.
- **Pooling:** Use 1-max pooling.
- **Regularization:** When increasing number of feature maps, try imposing stronger regularization, e.g. a dropout rate larger than 0.5.





## Conclusion

Before the rise of huge and complicated models using Transformer architecture, a simple CNN architecture with one layer of convolution can yeild excellent performance on sentence classification tasks. The model can take advantages of unsupervise pre-training of word vectors to improve overall performance. Improvements can be made in this architecture by increasing the number of CNN layers or utilizing sub-word model (using BPE tokenizer and fastText pretrained sub-word vectors). Because of its speed, we can use the CNN model as a strong baseline model before trying more complicated models such as BERT.

Thank you for staying with me to this point. If interested, you can check out other articles in my NLP tutorial series:
- [Tutorial: Fine-tuning BERT for Sentiment Analysis](https://chriskhanhtran.github.io/posts/bert_for_sentiment_analysis/)